In this section, we will demonstrate the case of developing a multi-step forecast model using a vector model

In [1]:
import numpy as np
from keras.models import Sequential
from keras.models import Model
from keras.layers import Dense
from keras.layers import Input
from keras.layers import Dense
from keras.layers.merge import concatenate

## Data Preparation

In [13]:
def split_sequence(series, n_step, pred_step):
    x_list, y_list = [], [] 
    for s in range(series.shape[0]-n_step-pred_step+1):
        x_list.append(series[s:s+n_step])
        y_list.append(series[s+n_step:s+n_step+pred_step])
    X, y = np.array(x_list), np.array(y_list)
    return X, y

In [21]:
def split_sequence_1(series, n_step, pred_step):
    X, y = np.array(), np.array()
    for s in range(series.shape[0]-n_step-pred_step+1):
        X = np.concatenate(X, [series[s:s+n_step]], axis=0)
        y = np.concatenate(y, [series[s+n_step:s+n_step+pred_step]], axis=0)
    return X, y

In [23]:
sequence = np.arange(10, 100, 10)
n_step = 3
pred_step = 2
X, y = split_sequence(sequence, n_step, pred_step)
for i in range(X.shape[0]):
    print(X[i], y[i])

[10 20 30] [40 50]
[20 30 40] [50 60]
[30 40 50] [60 70]
[40 50 60] [70 80]
[50 60 70] [80 90]


In [24]:
# define model
model = Sequential()
model.add(Dense(100, activation='relu', input_dim=n_step))
model.add(Dense(pred_step))
model.compile(optimizer='adam', loss='mse')

Instructions for updating:
Colocations handled automatically by placer.


In [25]:
model.fit(X, y, epochs=2000, verbose=0)

Instructions for updating:
Use tf.cast instead.


In [28]:
X.shape

(5, 3)

In [31]:
X_test = np.array([60, 70, 80])
print(X_test.shape)
X_test = np.array([[60, 70, 80]])
print(X_test.shape)

(3,)
(1, 3)


In [32]:
yhat = model.predict(X_test)
print(yhat)

[[ 90.00833 100.17039]]


## 7.5.1 Multiple Input Multi-step Output

In [38]:
in_seq1 = np.arange(10, 100, 10)
in_seq2 = np.arange(15, 105, 10)
out_seq = in_seq1 + in_seq2
print(out_seq)
in_seq1 = in_seq1.reshape(in_seq1.shape[0], 1)
in_seq2 = in_seq2.reshape(in_seq2.shape[0], 1)
out_seq = out_seq.reshape(out_seq.shape[0], 1)
print(out_seq)

[ 25  45  65  85 105 125 145 165 185]
[[ 25]
 [ 45]
 [ 65]
 [ 85]
 [105]
 [125]
 [145]
 [165]
 [185]]


In [40]:
dataset = np.hstack((in_seq1, in_seq2, out_seq))
print(dataset)

[[ 10  15  25]
 [ 20  25  45]
 [ 30  35  65]
 [ 40  45  85]
 [ 50  55 105]
 [ 60  65 125]
 [ 70  75 145]
 [ 80  85 165]
 [ 90  95 185]]


In [49]:
def split_sequence(series, n_step, pred_step):
    x_list, y_list = [], [] 
    for s in range(series.shape[0]-(n_step+pred_step-2)):
        x_list.append(series[s:s+n_step,:2])
        y_list.append(series[s+n_step-1:s+n_step-1+pred_step,2])
    X, y = np.array(x_list), np.array(y_list)
    return X, y

In [50]:
n_step = 3
pred_step = 2
X, y = split_sequence(dataset, n_step, pred_step)
print(X.shape, y.shape)
for i in range(X.shape[0]):
    print(X[i], y[i])

(6, 3, 2) (6, 2)
[[10 15]
 [20 25]
 [30 35]] [65 85]
[[20 25]
 [30 35]
 [40 45]] [ 85 105]
[[30 35]
 [40 45]
 [50 55]] [105 125]
[[40 45]
 [50 55]
 [60 65]] [125 145]
[[50 55]
 [60 65]
 [70 75]] [145 165]
[[60 65]
 [70 75]
 [80 85]] [165 185]


In [51]:
input_feature = X.shape[1] * X.shape[2]
X_flat = X.reshape(X.shape[0], input_feature)
print(X_flat)

[[10 15 20 25 30 35]
 [20 25 30 35 40 45]
 [30 35 40 45 50 55]
 [40 45 50 55 60 65]
 [50 55 60 65 70 75]
 [60 65 70 75 80 85]]


In [53]:
# define model
model = Sequential()
model.add(Dense(100, activation='relu', input_dim=input_feature))
model.add(Dense(pred_step))
model.compile(optimizer='adam', loss='mse')
model.fit(X_flat, y, epochs=2000, verbose=0)

In [54]:
x_test = np.array([[70, 75], [80, 85], [90, 95]])
x_test_flat = x_test.reshape(1, input_feature)
print(x_test_flat)

[[70 75 80 85 90 95]]


In [55]:
yhat = model.predict(x_test_flat)
print(yhat)

[[186.17825 206.37494]]


## 7.5.2 Multiple Parallel Input and Multi-step Output

In [57]:
print(dataset)

[[ 10  15  25]
 [ 20  25  45]
 [ 30  35  65]
 [ 40  45  85]
 [ 50  55 105]
 [ 60  65 125]
 [ 70  75 145]
 [ 80  85 165]
 [ 90  95 185]]


In [60]:
def split_sequence(series, n_step, pred_step):
    x_list, y_list = [], [] 
    for s in range(series.shape[0]-(n_step+pred_step)+1):
        x_list.append(series[s:s+n_step])
        y_list.append(series[s+n_step:s+n_step+pred_step])
    X, y = np.array(x_list), np.array(y_list)
    return X, y

In [63]:
n_step = 3
pred_step = 2
X, y = split_sequence(dataset, n_step, pred_step)
print(X.shape, y.shape)
for i in range(X.shape[0]):
    print(X[i], y[i])

(5, 3, 3) (5, 2, 3)
[[10 15 25]
 [20 25 45]
 [30 35 65]] [[ 40  45  85]
 [ 50  55 105]]
[[20 25 45]
 [30 35 65]
 [40 45 85]] [[ 50  55 105]
 [ 60  65 125]]
[[ 30  35  65]
 [ 40  45  85]
 [ 50  55 105]] [[ 60  65 125]
 [ 70  75 145]]
[[ 40  45  85]
 [ 50  55 105]
 [ 60  65 125]] [[ 70  75 145]
 [ 80  85 165]]
[[ 50  55 105]
 [ 60  65 125]
 [ 70  75 145]] [[ 80  85 165]
 [ 90  95 185]]


In [65]:
input_feature = X.shape[1] * X.shape[2]
output_feature = y.shape[1] * y.shape[2]
X_flat = X.reshape(X.shape[0], input_feature)
y_flat = y.reshape(y.shape[0], output_feature)
print(X_flat)

[[ 10  15  25  20  25  45  30  35  65]
 [ 20  25  45  30  35  65  40  45  85]
 [ 30  35  65  40  45  85  50  55 105]
 [ 40  45  85  50  55 105  60  65 125]
 [ 50  55 105  60  65 125  70  75 145]]


In [67]:
# define model
model = Sequential()
model.add(Dense(100, activation='relu', input_dim=input_feature))
model.add(Dense(output_feature))
model.compile(optimizer='adam', loss='mse')
model.fit(X_flat, y_flat, epochs=2000, verbose=0)

In [68]:
x_test = np.array([[60, 65, 125], [70, 75, 145], [80, 85, 165]])
x_test_flat = x_test.reshape(1, input_feature)
print(x_test_flat)

[[ 60  65 125  70  75 145  80  85 165]]


In [69]:
yhat = model.predict(x_test_flat)
print(yhat)

[[ 90.62378  96.18749 186.86935 100.93761 106.34216 208.01758]]


In [71]:
yhat.reshape(pred_step, y.shape[2])

array([[ 90.62378,  96.18749, 186.86935],
       [100.93761, 106.34216, 208.01758]], dtype=float32)